<a href="https://colab.research.google.com/github/ajw-42/arabic-nlp/blob/master/05_laser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Let's look at one more set of pre-trained word embeddings before moving on. They aren't necessarily a good fit for our problem, but they're an interesting and relatively new approach.

[LASER (Language-Agnostic SEntence Representations)](https://github.com/facebookresearch/LASER) is, as the name indicates, an approach for producing a single set of sentence-level representations across languages. This is great for multilingual datasets and also avoids the problem we encountered previously of having word vectors that we then averaged to get sentence- or document-level vecotrs. As we saw for `fasttext`, pre-trained embeddings are made available in addition to utilities for training your own embeddings from scratch.

The `LaserEmbeddings` library is a nice wrapper for `LASER` that makes the process of getting sentence representations a bit easier. In Google Colab, we'll need to pip install it before importing.



In [1]:
!pip install laserembeddings

     |████████████████████████████████| 51kB 5.5MB/s 
     |████████████████████████████████| 860kB 26.6MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.35-cp36-none-any.whl size=883999 sha256=5dc589b78f54dfdbde8550f06cf7b93a3ebbd247add2a258d195739a18ce92b3
  Stored in directory: /root/.cache/pip/wheels/63/2a/db/63e2909042c634ef551d0d9ac825b2b0b32dede4a6d87ddc94
Successfully built sacremoses


In [0]:
import pandas as pd
import numpy as np
import sklearn
import laserembeddings

In [3]:
from google.colab import drive

drive.mount('/content/gdrive')

train = pd.read_csv('gdrive/My Drive/RTANews_raw/arabic_train.csv')
val = pd.read_csv('gdrive/My Drive/RTANews_raw/arabic_val.csv')
test = pd.read_csv('gdrive/My Drive/RTANews_raw/arabic_test.csv')

train.head()

#Again, we'll limit ourselves to 20 classes for now.
train = train[train.label <= 20]
test = test[test.label <= 20]

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [4]:
#Now we need to download the pre-trained models
!python -m laserembeddings download-models



✅   Downloaded https://dl.fbaipublicfiles.com/laser/models/93langs.fcodes    
✅   Downloaded https://dl.fbaipublicfiles.com/laser/models/93langs.fvocab    
✅   Downloaded https://dl.fbaipublicfiles.com/laser/models/bilstm.93langs.2018-12-26.pt    

✨ You're all set!


We're also going to assume that each of our documents is one sentence. This isn't exactly true -- most are 3-4 sentences -- but it's a reasonable shortcut that will make things a bit easier here.

The `lang` argument is only used to determine how your text will be tokenized. For a multilingual dataset you could either not 

In [0]:
#This will take a while to run!
from laserembeddings import Laser

laser = Laser()

X_train = [laser.embed_sentences([text], lang='ar') for text in train.text]
X_test = [laser.embed_sentences([text], lang='ar') for text in test.text]


`embed_sentences()` outputs a two-dimensional array where the second dimension is 1, so we can flatten our results before we pass them into sklearn.


In [0]:
X_train = [np.concatenate(x) for x in X_train]
X_test = [np.concatenate(x) for x in X_test]

In [10]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

Y_train = train.label
Y_test = test.label

classifier = LogisticRegression(max_iter = 5000, multi_class='multinomial').fit(X_train, Y_train)
preds = classifier.predict(X_test)

f1_score(Y_test, preds, average = 'weighted')

0.7594143755886662

Not bad! It makes sense that this would perform slightly worse in our case than language-specific embeddings, but again LASER is a powerful tool for multi-lingual datasets.